In [ ]:
import json
import gspread
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from pydrive.auth import GoogleAuth

In [ ]:
# datetime.today()

In [ ]:
# Getting time index for the right sheet and columns
today = datetime.today() 
# + timedelta(days=2)
today_col = today.strftime("%a X%m/X%d").replace('X0','X').replace('X','')
month_sheet = today.strftime("%b %Y")
print(today_col)
print(month_sheet)
# today_col

In [ ]:
# Fetching the right sheets based on current month
#gc = gspread.service_account(filename="C:\\Users\\MSI PC\\Documents\\CPTracking\\credentials.json")
sheet = gc.open('CRM | Comm Calendar')
# Defining push, pop-up, inbox sheets?
push_sheet = sheet.worksheet(month_sheet + "_Push")
popup_sheet = sheet.worksheet(month_sheet + "_Popup")
inbox_sheet = sheet.worksheet(month_sheet + "_Inbox")

In [ ]:
type(push_sheet)

In [ ]:
header_indices = push_sheet.col_values(2)
row_indices = [i for i, x in enumerate(header_indices) if not x == ""]
col_index = push_sheet.find(today_col).col
value_list = push_sheet.col_values(col_index)
data_list = [value_list[i] for i in row_indices]
data_list


In [ ]:
len(value_list)

In [ ]:
# Splitting number of columns by the number of cities to convert list into df
city_count = sum(map(lambda x : x == 'city', header_indices))

df = pd.DataFrame.from_records(np.array_split(data_list[1:], city_count))

In [ ]:
header_list = [header_indices[i] for i in row_indices]
header_columns = np.array_split(header_list[1:], city_count)[0]
format_header_columns = [x.lower().replace(" ", "_") for x in header_columns]
df.columns = format_header_columns

In [ ]:
# Creating new time of day column and removing waiting_list
i = 0

while i < len(df.city):
    df.loc[i, 'time_of_day'] = "Morning"
    i += 1
    df.loc[i, 'time_of_day'] = "Afternoon"
    i += 1
    df.loc[i, 'time_of_day'] = "Evening"
    i += 1
    df = df.drop([i])
    i += 1

In [ ]:
# Cleaning df: Removing Manual push notifications and all empty content pushes
df = df[(df.content != '') & ( ~df.targeted_users.str.contains('Manual'))]

In [ ]:
df

In [ ]:
df['campaign_name'] = df.city + df.time_of_day + "Push"

In [ ]:
# braze_api_key = '7d9eb688-d3a1-4fe6-a43b-cffd1ecf7e53' #dev environment
braze_api_key = 'a2504643-c098-41a3-9e3b-d1d3ab116b93' #product environment

In [ ]:
# Getting df of campaign ids of interest

payload={}
headers = {
  'Authorization': 'Bearer ' + braze_api_key
}

## The page of campaigns to return, defaults to 0 (returns the first set of up to 100) 
## Currently, there's only 3 pages in braze so I leave it at 4 just in case we decide to expand in the future.
pages = 4 

campaign_list = []

for i in range(pages):

    url = "https://rest.iad-03.braze.com/campaigns/list?page="+ str(i) +"&include_archived=false&sort_direction=desc"

    response = requests.request("GET", url, headers=headers, data=payload)

    cleaned_res = json.loads(response.text)

    campaign_list.append(cleaned_res['campaigns'])

campaign_list = list(np.concatenate(campaign_list).flat)
campaign_df = pd.DataFrame()
campaign_tags = ('Push', 'Inbox')
campaign_df['campaign_id'] = [i['id'] for i in campaign_list if (str(i['name']).endswith(campaign_tags) & i['is_api_campaign'] == True)]
campaign_df['campaign_name'] = [i['name'] for i in campaign_list if (str(i['name']).endswith(campaign_tags) & i['is_api_campaign'] == True)]


In [ ]:
campaign_df

In [ ]:
# Getting df of segment ids of interest

payload={}
headers = {
  'Authorization': 'Bearer ' + braze_api_key
}

segment_list = []

## pages already defined above, used to overcome the 100 results per page return limit by braze
for i in range(pages):

    url = "https://rest.iad-03.braze.com/segments/list?page="+ str(i) +"&sort_direction=desc"

    response = requests.request("GET", url, headers=headers, data=payload)

    cleaned_res = json.loads(response.text)

    segment_list.append(cleaned_res['segments'])

segment_list = list(np.concatenate(segment_list).flat)
segment_dict = [dictionary for dictionary in segment_list if ('API' in dictionary["tags"] and 'PRODUCTION' in dictionary["tags"])]
segment_df = pd.DataFrame.from_dict(segment_dict)
segment_df.rename(columns={"id":"segment_id", "name":"segment_name", "tags":"segment_tags"}, inplace=True)

In [ ]:
segment_df

In [ ]:
# Getting df of variation ids for android and ios push based on campaign of interest
def get_variation_id(campaign_id):
        
    url = "https://rest.iad-03.braze.com/campaigns/details?campaign_id=" + str(campaign_id)

    payload={}
    headers = {
      'Authorization': 'Bearer ' + braze_api_key
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    cleaned_res = json.loads(response.text)
    
    lt = [] 
    lt.append(cleaned_res.get('name', None))
    lt.append(list(cleaned_res['messages'].keys())[0])
    lt.append(list(cleaned_res['messages'].keys())[1])
    
    return lt

In [ ]:
lt = [get_variation_id(i) for i in campaign_df.campaign_id]

campaign_details_df = pd.DataFrame(lt, columns=['campaign_name', 'ios_variation_id', 'android_variation_id'])

In [ ]:
push_df = df.merge(campaign_details_df.merge(campaign_df, on='campaign_name'), on='campaign_name')\
    .merge(segment_df, left_on='city', right_on='segment_name')

In [ ]:
push_df['braze_api_key'] = braze_api_key
push_df['schedule_time'] = today.strftime("%Y-%m-%d")+'T'+push_df['time']+":00"

In [ ]:
push_df

In [ ]:
## Test segment functionality
# test_push_df = push_df
# test_push_df['segment_id'] = '8a5e504b-f0af-4fef-a357-683aacf04c93' # for segment Test_CRM_Automate 
# push_df['segment_id'] = 'eed772fa-00d3-4539-bcdc-d305324b5688' # for segment Test_CRM_Automate 
# test_push_df.loc[test_push_df.city == 'HCM','segment_id']  = '1b369b49-6be1-4172-abea-e0c1a780918d'
# test_push_df.loc[test_push_df.city == 'HN','segment_id']  = 'ad62af46-2d20-4aee-8870-0b5016b3d146'

## Test push time
# test_push_df = push_df[(push_df.city == 'HCM') | (push_df.city == 'HN')]
# test_push_df = push_df[push_df.time == "14:00"]

# test_push_df
# .reset_index(drop=True, inplace=True)

In [ ]:
# test_push_df

In [ ]:
##Function to print out schedule post payload data 

# def print_payload(api_key, campaign_id, segment_id, time, title, alert, custom_uri, ios_vid, android_vid):
    
#     payload = {
#           "api_key": api_key,
#           "campaign_id": campaign_id,
#           "broadcast": True,
#           "segment_id": segment_id,
#           "schedule": { 
#             "time": time,
#             "in_local_time": True
#           },
#           "override_frequency_capping": True,
#           "messages": {
#              "apple_push": {
#              "alert": {
#                  "title": title,
#                  "body": alert},
#              "custom_uri": custom_uri,
#              "message_variation_id": ios_vid
#              },
#              "android_push": {
#              "title": title,
#              "alert": alert,
#              "custom_uri": custom_uri,
#              "message_variation_id": android_vid
#              }
#            }
#         }

#     print(payload)

In [ ]:
# # Printing out all scheduled posts
# [print_payload(push_df.braze_api_key[i], push_df.campaign_id[i], push_df.segment_id[i], push_df.schedule_time[i], \
#                     push_df.title[i], push_df.content[i], push_df.deep_link[i], push_df.ios_variation_id[i], \
#                     push_df.android_variation_id[i]) for i in range(len(push_df))]

In [ ]:
def post_push_campaign(api_key, campaign_id, segment_id, time, title, alert, custom_uri, ios_vid, android_vid):
    
    url = "https://rest.iad-03.braze.com/messages/schedule/create"

    payload = {
          "api_key": api_key,
          "campaign_id": campaign_id,
          "broadcast": True,
          "segment_id": segment_id,
          "schedule": { 
            "time": time,
            "in_local_time": True
          },
          "override_frequency_capping": True,
          "messages": {
             "apple_push": {
             "alert": {
                 "title": title,
                 "body": alert},
             "custom_uri": custom_uri,
             "message_variation_id": ios_vid
             },
             "android_push": {
             "title": title,
             "alert": alert,
             "custom_uri": custom_uri,
             "message_variation_id": android_vid
             }
           }
        }
    
    payload = json.dumps(payload)
    
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload.encode('utf-8'))

    print(response.text)

In [ ]:
# ## Testing on our end to make sure that users receive all pushes 
# [post_push_campaign(test_push_df.braze_api_key[i], test_push_df.campaign_id[i], test_push_df.segment_id[i], test_push_df.schedule_time[i], \
#                     test_push_df.title[i], test_push_df.content[i], test_push_df.deep_link[i], test_push_df.ios_variation_id[i], \
#                     test_push_df.android_variation_id[i]) for i in range(len(test_push_df))]

In [ ]:
# #Actually Sending pushes to end-users 
# [post_push_campaign(push_df.braze_api_key[i], push_df.campaign_id[i], push_df.segment_id[i], push_df.schedule_time[i], \
#                    push_df.title[i], push_df.content[i], push_df.deep_link[i], push_df.ios_variation_id[i], \
#                    push_df.android_variation_id[i]) for i in range(len(push_df))]

In [ ]:
# Flagging date as push scheduled 
push_sheet.update_cell(1, col_index, 'SCHEDULED')

## Popup and Inbox section (Under Construction!!)

In [ ]:
ib_header_indices = inbox_sheet.col_values(2)
ib_row_indices = [i for i, x in enumerate(ib_header_indices) if not x == ""]
ib_col_index = inbox_sheet.find(today_col).col
ib_value_list = inbox_sheet.col_values(ib_col_index)
ib_data_list = [ib_value_list[i] for i in ib_row_indices]
ib_data_list

In [ ]:
# GDrive authorization for downloading images
http://localhost:8888/notebooks/Documents/CRM_API/braze_api.ipynb#Popup-and-Inbox-section-(Under-Construction!!)
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()
# drive = GoogleDrive(gauth)

In [ ]:
# file_obj = drive.CreateFile({'id': '1XlSmsaao9Y4K1fxzcwVvbUcncWVuX1yd'})
# file_obj.GetContentFile('Demo.jpg')